<a href="https://colab.research.google.com/github/acollin19/comp551/blob/Angele/A2/assignment2_group_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2

## Process:

### Task 1: Data Processing  


1. IMDB Reviews
2. 20 news groups: a multi-class labelled textual dataset


### Task 2: Implement Logistic and Multiclass classifiers

1. Logistic Regression
2. Multiclass Regression
3. Comparisons using ROC curve


### Task 3: Running Experiments



1.    Most positive/negative Z-scores
2.   Implement:  
    * Binary classification on the IMDB Reviews

  *   Multi-class classification on the 20 news group dataset

3. Binary classification AUROC on IMDB data
  *   Logistic Regression 
  *   KNN

4. Multiclass classification accuracy

5. Compare Accuracy of Models (Plot)







In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_svmlight_file
from scipy.sparse import csr_matrix
import scipy.sparse as sp


from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Multiclass Regression and ROC curve comparisons
from sklearn import model_selection
from matplotlib import pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from nltk import word_tokenize, download
from nltk.corpus import stopwords


In [2]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz
!ls aclImdb
!ls aclImdb/test
!ls aclImdb/train


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  11.5M      0  0:00:06  0:00:06 --:--:-- 18.8M
imdbEr.txt  imdb.vocab	README	test  train
labeledBow.feat  neg  pos  urls_neg.txt  urls_pos.txt
labeledBow.feat  pos	unsupBow.feat  urls_pos.txt
neg		 unsup	urls_neg.txt   urls_unsup.txt


In [99]:
# IMDB dataset preprocession
IMDB_X_train, IMDB_y_train= load_svmlight_file('aclImdb/train/labeledBow.feat')
IMDB_X_test, IMDB_y_test = load_svmlight_file('aclImdb/test/labeledBow.feat') 


# Task 1 : Data processing



## Task 1.1 : IMDB

 - X contain features based on the words in the movie reviews
 - y contain labels for whether the review score

In [100]:
# 1st element in tuple represents the file its in, 2nd element represents the index of word it represents in vocab, float represents frequency
# want to remove the words that appear in less than 1% of all documents # only appear in 250 documents
# want to remove the words that appear in more than 50% of all documents # appear in 12500 documents
# sign = sum(IMDB_X_train.sign()) # 1 if the word appears in the text 0 if not
# good = ((sign < 12500) - (sign < 250))
# indices = sp.find(good)

sign = IMDB_X_train.sign()
num_file_occurences_train = np.asarray(sign.sum(axis=0))# number of files each word appears in
#ndarray of the indices of the words excluding stop words and rare words
indices = np.intersect1d((np.flatnonzero(12500 > num_file_occurences_train)), (np.flatnonzero(250 < num_file_occurences_train)))


# Trim dataset to indices
IMDB_X_train = IMDB_X_train[:,indices]
IMDB_X_test = IMDB_X_test[:,indices]
IMDB_X_train.todense()

matrix([[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 2., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.]])

## Z-Score Function

In [5]:
# from sklearn.utils.extmath import sparse
# # Z - Score
# def zscore(X,y):

#   N = X.shape[0]
#   std = (y - y.mean()) / y.std()
#   z = sp.csr_matrix.dot(sp.csr_matrix.transpose(X),std)/ np.sqrt(N)
#   absZ = np.abs(z)
#   return (absZ,z)

# zscores, noAbsZ = zscore(IMDB_X_train,IMDB_y_train)
# # highestzindex = np.argsort(zscores)[-200:][::-1] # top 200 highest z scores
# # lowestzindex = np.argsort(zscores)[:200][::1] # worst 200 highest z scores
# # zindex = np.concatenate((highestzindex, lowestzindex))

# #choose by their absolute value
# zindex = np.argsort(noAbsZ)[::-1][:300]
# zindex_notAbs = np.argsort(noAbsZ)[::-1][:300]
# #words in zindex
# wordsZ = noAbsZ[np.argsort(noAbsZ)[::-1][:300]]


# # get correct indexed feature set
# new_IMDB_X_train = IMDB_X_train[:,zindex].toarray()
# new_IMDB_X_test = IMDB_X_test[:,zindex].toarray()


# data = open('aclImdb/imdb.vocab')
# vocab = data.read().split('\n')

# # get correct indexed word list
# word_list = [vocab[i]for i in indices]

# word_list = [word_list[i] for i in zindex]

# posindexesZ = np.argsort(zindex)[-10:][::-1] # highest-lowest
# negindexesZ = np.argsort(zindex)[:10][::1] # lowest-highest

# pos = [word_list[i] for i in posindexesZ]
# neg = [word_list[i] for i in negindexesZ]


# print((noAbsZ))
# print(len(word_list))
# print(pos)
# print(noAbsZ[np.argsort(noAbsZ)[-10:][::-1]])
# print(neg)
# print(noAbsZ[np.argsort(noAbsZ)[:10][::1]])

In [104]:
## Test 
from sklearn.utils.extmath import sparse
# Map in df 
data = open('aclImdb/imdb.vocab')
vocab = data.read().split('\n')

# get correct indexed word list
word_list = [vocab[i]for i in indices]

# Z - Score
def zscore(X,y):

  N = X.shape[0]
  std = (y - y.mean()) / y.std()
  z = sp.csr_matrix.dot(sp.csr_matrix.transpose(X),std)/ np.sqrt(N)
  absZ = np.abs(z)
  return (absZ,z)

zscores, noAbsZ = zscore(IMDB_X_train,IMDB_y_train)

data = list(zip(zscores,noAbsZ,word_list))
df = pd.DataFrame.from_records(data, columns =['zscore', 'nonAbsZscore', 'word'])
df['indices'] = df.index

# get between [100,1000] range
df = df.loc[101:400:1]
zindices = df['indices'].tolist()

# get correct indexed feature set
# IMDB_X_train = IMDB_X_train[:,zindices].toarray()
# IMDB_X_test = IMDB_X_test[:,zindices].toarray()


word_list = [word_list[i] for i in zindices]

imdb_X_train = IMDB_X_train[:,zindices].toarray()
imdb_X_test = IMDB_X_test[:,zindices].toarray()

imdb_y_test = [IMDB_y_test.tolist()[i] for i in zindices]
imdb_y_train = [IMDB_y_train.tolist()[i] for i in zindices]



(25000, 300)
300


        zscore  nonAbsZscore      word  indices
213  15.873323    -15.873323     worst      213
129  10.515839    -10.515839   nothing      129
106  10.054247    -10.054247       why      106
341   9.249509     -9.249509     awful      341
400   8.797149     -8.797149     waste      400
198   8.324481     -8.324481   minutes      198
346   7.993304     -7.993304  terrible      346
122   7.756602     -7.756602     thing      122
336   7.634698     -7.634698    stupid      336
317   7.155281     -7.155281    boring      317
       zscore  nonAbsZscore         word  indices
182  5.605140      5.605140       family      182
177  5.977714      5.977714       always      177
202  6.116688      6.116688  performance      202
152  6.285921      6.285921        young      152
147  6.292276      6.292276        world      147
270  6.345468      6.345468    beautiful      270
163  6.584429      6.584429         both      163
123  6.690121      6.690121        years      123
342  7.337024      7.3

In [ ]:
# Loading 20 news group dataset
categories=['rec.sport.hockey','sci.electronics','talk.politics.guns','sci.space']
twenty_train = fetch_20newsgroups(subset='train', categories=categories, remove=(['headers', 'footers', 'quotes']), shuffle=True, random_state=42)

'''
Cleaning: 
- filter out rare words, stopwords, and words that are not relevant to any of the 4 class labels.
- MI to select the top M ∈ [10, 100] feature words per class and take the union of all top feature words to train your multiclass model.

'''

download('punkt') #tokenizer, run once
download('stopwords') #stopwords dictionary, run once

# Tokenize, remove stopwords
stop_words = stopwords.words('english')

def preprocess(text):
    text = text.lower()
    doc = word_tokenize(text)
    doc = [word for word in doc if word not in stop_words]
    # Restricts string to alphabetic characters only
    doc = [word for word in doc if word.isalpha()] 
    return doc

# text and ground truth labels
texts, y = twenty_train.data, twenty_train.target

corpus = [preprocess(text) for text in texts]


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Task 1.2 : Twenty Train

Text preprocessing, tokenizing and filtering of stopwords are all included in the CountVectorizer function


In [ ]:
# Turn the text content into feature vectors
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) # builds a dict of features and transforms docs to feature vectors
X_train_counts.shape

# Occurences to frequences : TF_IDF

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [ ]:
X = twenty_train.data # Test content

# Turn the text content into feature vectors
count_vect = CountVectorizer() 
X_train_counts = count_vect.fit_transform(twenty_train.data) # builds a dict of features and transforms docs to feature vectors
X_train_counts.shape

# Occurences to frequences : TF_IDF

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X = X_train_tfidf
X.todense()
twenty_train_arr = X.toarray()

y_cat = twenty_train.target
# convert array of class indices to one-hot encoded array
y = np.zeros((y_cat.size, y_cat.max() + 1))
y[np.arange(y_cat.size), y_cat] = 1

#print(zscore(twenty_train_arr))

# Task 2 : Implementing Classes

## Task 2.1 : Logistic Regression Class

In [105]:
# Logistic function
z = np.linspace(-5,5,100)
logistic = lambda z: 1./ (1 + np.exp(-z))

# From class Logistic Regression collab
class LogisticRegression:
    
    def __init__(self, add_bias=True, learning_rate=.1, epsilon=1e-4, max_iters=1e5, verbose=False):
        self.add_bias = add_bias
        self.learning_rate = learning_rate
        self.epsilon = epsilon                        #to get the tolerance for the norm of gradients 
        self.max_iters = max_iters                    #maximum number of iteration of gradient descent
        self.verbose = verbose

    def gradient(self, x, y):
        N,D = x.shape
        yh = logistic(np.dot(x, self.w))    # predictions  size N
        grad = np.dot(x.T, yh - y)/N        # divide by N because cost is mean over N points
        return grad
        
    def fit(self, x, y):
        if x.ndim == 1:
            x = x[:, None]
        if self.add_bias:
            N = x.shape[0]
            x = np.column_stack([x,np.ones(N)])
        N,D = x.shape
        self.w = np.zeros(D)
        g = np.inf 
        t = 0
        # the code snippet below is for gradient descent
        while np.linalg.norm(g) > self.epsilon and t < self.max_iters:
            g = self.gradient(x, y)
            self.w = self.w - self.learning_rate * g 
            t += 1
        
        if self.verbose:
            print(f'terminated after {t} iterations, with norm of the gradient equal to {np.linalg.norm(g)}')
            print(f'the weight found: {self.w}')
        return self
    
    def predict(self, x):
        if x.ndim == 1:
            x = x[:, None]
        Nt = x.shape[0]
        if self.add_bias:
            x = np.column_stack([x,np.ones(Nt)])
        yh = logistic(np.dot(x,self.w))            #predict output
        return yh

 

### Evaluation using Logistic Regression


In [106]:
from sklearn.metrics import accuracy_score

model = LogisticRegression(verbose=False, add_bias=False, max_iters=1e3,learning_rate=.15, epsilon=1e-3)
fit = model.fit(imdb_X_train,imdb_y_train)

y_train_pred = fit.predict(imdb_X_train)
y_test_pred = fit.predict(imdb_X_test)

y_pred = model.predict(imdb_X_test)
score = accuracy_score(imdb_y_test, y_pred)
score

ValueError: ignored

In [97]:
# Applying Logistic Regression on the IMDB data

model = LogisticRegression(max_iters=1e3,learning_rate=.1, epsilon=1e-3)
fit = model.fit(IMDB_X_train,IMDB_y_train)

y_train_pred = fit.predict(IMDB_X_train)
y_test_pred = fit.predict(IMDB_X_test)

# Threshold probabilities
y_train_pred = (y_train_pred > 0.5).astype(int)
y_test_pred = (y_test_pred > 0.5).astype(int)

# Accuracy = correctly classified / total classified
acc_train = sum(y_train_pred==IMDB_y_train)/len(IMDB_y_train)
acc_test = sum(y_test_pred==IMDB_y_test)/len(IMDB_y_test)
print(f"train accuracy: {acc_train:.3f}; test accuracy: {acc_test:.3f}")

# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(IMDB_y_test,y_test_pred)

# Gradient
gradient = model.gradient(IMDB_X_train, IMDB_y_train) 
gradient

ValueError: ignored

In [ ]:
from sklearn.model_selection import cross_val_score, cross_val_predict
# Cross Validation Scores
clf_lr = sk_LogisticRegression()
scores = cross_val_score(clf_lr, new_IMDB_X_train, IMDB_y_train, cv=10, n_jobs=4)
scores

## Task 2.2 : Multiclass Regression Class

In [ ]:
# From class Multiclass Regression collab
class Multinomial_logistic:
    def __init__(self, nFeatures, nClasses):
        self.W = np.random.rand(nFeatures, nClasses)

    def predict(self, X):
        y_pred = np.exp(np.matmul(X, self.W))
        return y_pred / y_pred.sum(axis=1).reshape(X.shape[0], 1)

    def grad(self, X, y):
        return np.matmul(X.transpose(), self.predict(X) - y)

    def ce(self, X, y):
        return -np.sum(y * np.log(self.predict(X)))

    # modify it to add stopping criteria (what can you think of?)
    def fit(self, X, y, X_valid=None, y_valid=None, lr=0.005, niter=100):
        losses_train = np.zeros(niter)
        losses_valid = np.zeros(niter)
        for i in range(niter):
            self.W = self.W - lr * self.grad(X, y)
            loss_train = self.ce(X, y)
            losses_train[i] = loss_train
            if X_valid is not None and y_valid is not None:
                loss_valid = self.ce(X_valid, y_valid)
                losses_valid[i] = loss_valid
                print(f"iter {i}: {loss_train:.3f}; {loss_valid:.3f}")
            else:
                print(f"iter {i}: {loss_train:.3f}")
        return losses_train, losses_valid

    def check_grad(self, X, y):
        N, C = y.shape
        D = X.shape[1]

        diff = np.zeros((D, C))

        W = self.W.copy()

        for i in range(D):
            for j in range(C):
                epsilon = np.zeros((D, C))
                epsilon[i, j] = np.random.rand() * 1e-4

                self.W = self.W + epsilon
                J1 = self.ce(X, y)
                self.W = W

                self.W = self.W - epsilon
                J2 = self.ce(X, y)
                self.W = W

                numeric_grad = (J1 - J2) / (2 * epsilon[i, j])
                derived_grad = self.grad(X, y)[i, j]

                diff[i, j] = np.square(derived_grad - numeric_grad).sum() / \
                             np.square(derived_grad + numeric_grad).sum()

        # print(diff)
        return diff.sum()

    def evaluate(self, y, y_pred):
      accuracy = sum(y_pred.argmax(axis=1) == y.argmax(axis=1))
      accuracy = accuracy / y.shape[0]

      return accuracy

### Evaluation using Multiclass Regression

In [ ]:
# Applying Multiclass Regression on the 4-class prediction from the 20-news-group data

# Split the data into training, validation, and testing
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X, y, test_size = 0.33, random_state=1, shuffle=True)

X_train, X_valid, y_train, y_valid = model_selection.train_test_split(
    X_train, y_train, test_size = 0.5, random_state=1, shuffle=True)

#Create model object
D = X.shape[1]
C = y.shape[1]

mlr = Multinomial_logistic(D, C)

# check grad
print(mlr.check_grad(X_train, y_train))



## Task 2.3 : Method comparison using ROC curve

In [ ]:
models = [KNeighborsClassifier(),
          DecisionTreeClassifier(),
          sk_LogisticRegression()]

perf = {}

# our implementation is slow you may try sklearn version
logitreg = sk_LogisticRegression()
fit = logitreg.fit(new_IMDB_X_train, IMDB_y_train)
y_test_prob = fit.predict(new_IMDB_X_test)
fpr, tpr, _ = roc_curve(IMDB_y_test, y_test_prob)
auroc = roc_auc_score(IMDB_y_test, y_test_prob)
perf["LogisticRegression (ours)"] = {'fpr':fpr, 'tpr':tpr, 'auroc':auroc}

for model in models:
    fit = model.fit(new_IMDB_X_train, IMDB_y_train)
    y_test_prob = fit.predict_proba(new_IMDB_X_test)[:,1]
    fpr, tpr, _ = roc_curve(IMDB_y_test, y_test_prob)
    auroc = roc_auc_score(IMDB_y_test, y_test_prob)
    if type(model).__name__ == "LogisticRegression":
        perf["LogisticRegression (sklearn)"] = {'fpr':fpr,'tpr':tpr,'auroc':auroc}
    else:
        perf[type(model).__name__] = {'fpr':fpr,'tpr':tpr,'auroc':auroc}


plt.clf()
i = 0
for model_name, model_perf in perf.items():
    plt.plot(model_perf['fpr'], model_perf['tpr'],label=model_name)
    plt.text(0.4, i+0.1, model_name + ': AUC = '+ str(round(model_perf['auroc'],2)))
    i += 0.1

plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title('ROC in predicting Titanic survivor')
plt.legend(bbox_to_anchor=(1.04, 0.5), loc="upper left")
plt.show()
# plt.savefig("roc_curve.png", bbox_inches='tight', dpi=300)
# plt.close()

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


ValueError: ignored

# Task 3 : Running Experiments

## Task 3.1 : Z-Scores
Report top 10 features with most positive/negative z-scores on the IMDB data using simple linear regression on the movie rating scores.  

QUESTION TO ANSWER: Do they make sense for calling a movie good and bad, respectively?


ANSWER: The top features with the most positive z-scores do make sense as it includes words like 'family', 'beutifully','excellent','wonderful' which are all generally used with a positive connotation. 
Likewise, the top features with the most negative make sense with calling a movie bad as it includes words such as 'worst', 'terrible', 'awful', 'stupid' which are all associated with negative connotation and can imply that a movie is bad.




In [112]:
negwords = df.sort_values('nonAbsZscore').head(10)
poswords = df.sort_values('nonAbsZscore').tail(10)

print(poswords)
print(negwords)




       zscore  nonAbsZscore         word  indices
182  5.605140      5.605140       family      182
177  5.977714      5.977714       always      177
202  6.116688      6.116688  performance      202
152  6.285921      6.285921        young      152
147  6.292276      6.292276        world      147
270  6.345468      6.345468    beautiful      270
163  6.584429      6.584429         both      163
123  6.690121      6.690121        years      123
342  7.337024      7.337024    wonderful      342
282  8.154902      8.154902    excellent      282
        zscore  nonAbsZscore      word  indices
213  15.873323    -15.873323     worst      213
129  10.515839    -10.515839   nothing      129
106  10.054247    -10.054247       why      106
341   9.249509     -9.249509     awful      341
400   8.797149     -8.797149     waste      400
198   8.324481     -8.324481   minutes      198
346   7.993304     -7.993304  terrible      346
122   7.756602     -7.756602     thing      122
336   7.634698    

## Extra Experiemnts: Linear Regression for predicting ratings in the IMDB data

In [ ]:
from sklearn.linear_model import LinearRegression
LinR = LinearRegression()
  
LinR.fit(new_IMDB_X_train, IMDB_y_train)
print(LinR.score(new_IMDB_X_test, IMDB_y_test))

true_w = -4
plt.clf()
plt.plot(new_IMDB_X_train, IMDB_y_train, '.')
w_closedform = model.w
plt.plot(new_IMDB_X_train, true_w*new_IMDB_X_train, 'b-', label='true model')
plt.plot(new_IMDB_X_train, new_IMDB_X_train*w_closedform, 'g--', label='closed-form')
plt.xlabel('x')
plt.ylabel(r'$y=xw$')
plt.legend()
# plt.savefig("linear_fit_closedform.png",bbox_inches='tight', dpi=300)



0.4363111581143996


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  app.launch_new_instance()


KeyboardInterrupt: ignored

Error in callback <function flush_figures at 0x7eff5f589200> (for post_execute):


KeyboardInterrupt: ignored